# If it is helpful, Please upvote!!!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load Data

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
len(train), len(test)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# Check if there are columns with null values
for col in test.columns:
    na_tr = len(train[train[col].isna()])
    na_te = len(test[test[col].isna()])
    print(col, na_tr, na_te)

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
# import tensorflow_addons as tfa
import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *

In [ ]:
feat_cols = [f'cont{i}' for i in range(1, 15)]
feat_cols

In [ ]:
all_feat_cols = feat_cols

# StandardScaler

In [ ]:
scalar = StandardScaler()
train[all_feat_cols] = scalar.fit_transform(train[all_feat_cols])
test[all_feat_cols] = scalar.transform(test[all_feat_cols])

# Define RMSE loss function

In [ ]:
# loss function
def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

# Define Model

In [ ]:
# model function
def build_model():
    inp = Input(shape=(len(all_feat_cols)))
    x = Dense(64, activation='relu')(inp)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='linear')(x)

    model = Model(inputs=inp, outputs=output)

    adam = tf.optimizers.Adam(lr=1e-3)
    # radam = tfa.optimizers.RectifiedAdam(learning_rate=LR, weight_decay=1e-5)
    # lookahead = tfa.optimizers.Lookahead(radam, sync_period=5, slow_step_size=0.5)
    model.compile(optimizer=adam, loss=RMSE)

    return model

In [ ]:
def RMSE_fn(y_true, y_pred):
    return np.sqrt(np.mean(np.power(np.array(y_true, float).reshape(-1, 1) - np.array(y_pred, float).reshape(-1, 1), 2)))

# Train Model

In [ ]:
cv_rmse = []
oof_pred = np.zeros(len(train))
test_pred = np.zeros(len(test))

import time
start_time = time.time()
kf = KFold(5, shuffle=True, random_state=42)
for _fold, (train_idx, valid_idx) in enumerate(kf.split(train[all_feat_cols].values, train['target'].values)):
    print(f'\nFold{_fold} Train/Val: {len(train_idx)}/{len(valid_idx)}')
    train_feat, valid_feat = train.loc[train_idx, all_feat_cols].values, train.loc[valid_idx, all_feat_cols].values
    train_label, valid_label = train.loc[train_idx].target.values, train.loc[valid_idx].target.values
    
    K.clear_session()
    model = build_model()
    model_weights = f'./model{_fold}.h5'
    checkpoint = ModelCheckpoint(model_weights, monitor='val_loss', verbose=0, save_best_only=True, mode='min',
                                 save_weights_only=True)
#     plateau = ReduceLROnPlateau()
    plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_delta=1e-4, mode='min')
    early_stopping = EarlyStopping(monitor="val_loss", patience=5)
    history = model.fit(train_feat, train_label,
                        validation_data=(valid_feat, valid_label),
                        batch_size=128,
                        epochs=100,
                        callbacks=[plateau, checkpoint, early_stopping],
                        verbose=2
                       )

    val_rmse = min(history.history['val_loss'])
    print(f'Fold{_fold} val_rmse: {val_rmse:.4f}\tTime={(time.time()-start_time) / 60:.2f}min')
    cv_rmse.append(val_rmse)
    
    oof_pred[valid_idx] = model.predict(valid_feat).reshape(-1)
    test_pred += model.predict(test[all_feat_cols].values).reshape(-1) / 5

cv_score = np.mean(cv_rmse)
oof_score = RMSE(train.target.values, oof_pred)
print(f'cv_score: {cv_score:.4f}')
print(f'oof_score: {oof_score:.4f}')

In [ ]:
cv_score: 0.7108
oof_score: 0.7119

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_test.png',show_shapes=True)

# Submit

In [ ]:
output = pd.DataFrame({"id":test.id, "target":test_pred})
output.to_csv('submission.csv', index=False)

In [ ]:
output.target.min() / 5, output.target.max() / 5

# Just Upvoting!!!